<a href="https://colab.research.google.com/github/pantipolo12/Diet-Recommendation-LLM/blob/main/IR_PROJECT_Diet_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OCR (Optical Character Recognition)

In [1]:
# @title
# Given Dataset (Chose relevant prescriptions)
!gdown 1DvcXqFzEDlrFQTOQV_qEZjDWBT1MnnFl
!gdown 1ENdiOHRYL8ImpFuBR9nLSRohF8nBmvJS
!gdown 1rHIa19GiUb5xlG6ptdJXyaTs5GD8tyl0
!gdown 196Ww4pmzDvRHNIhBU4zhwfO9cY7qQJI0

Downloading...
From: https://drive.google.com/uc?id=1DvcXqFzEDlrFQTOQV_qEZjDWBT1MnnFl
To: /content/prescription 5.pdf
100% 143k/143k [00:00<00:00, 12.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1ENdiOHRYL8ImpFuBR9nLSRohF8nBmvJS
To: /content/prescription 6.pdf
100% 296k/296k [00:00<00:00, 5.30MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rHIa19GiUb5xlG6ptdJXyaTs5GD8tyl0
To: /content/prescription 7.pdf
100% 255k/255k [00:00<00:00, 7.62MB/s]
Downloading...
From: https://drive.google.com/uc?id=196Ww4pmzDvRHNIhBU4zhwfO9cY7qQJI0
To: /content/prescription 8.pdf
100% 338k/338k [00:00<00:00, 4.55MB/s]


In [2]:
# Installing OCR libraries
%%capture
!pip install pypdfium2
!apt-get install poppler-utils
!pip install pdf2image
!apt install tesseract-ocr
!apt install libtesseract-dev
!pip install pytesseract

In [3]:
# Importing necessary libraries
import pypdfium2 as pyp
from pdf2image import convert_from_path
from IPython.display import display, Image
import cv2
import pytesseract
import os
import pandas as pd
import numpy as np
import nltk
import re
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Removes any skew in the image
def deskew(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.bitwise_not(gray)
    coords = np.column_stack(np.where(gray > 0))
    angle = cv2.minAreaRect(coords)[-1]

    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle

    (h, w) = image.shape[:2]
    center = (w // 2, h // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    return rotated

# Extracts text from image
def extract_text_from_image(image):
    text = pytesseract.image_to_string(image)
    return text

In [5]:
# Directory where the pdfs are stored
data_dir = '/content'

In [7]:
# Returns tokenized text extracted from pdf or images after new line/empty string removal
def read_prescription(path):
  pdf = pyp.PdfDocument(path)
  pages = len(pdf)
  text = []

  for i in range(pages):
      raw = pdf[i].get_textpage().get_text_range()
      words = word_tokenize(raw)
      text.extend(words)

  # If the pdf is not read, that means we have an image and we read via OCR
  if len(text) == 0:
    pages = convert_from_path(path)
    extracted_text = []

    for page in pages:
        preprocessed_image = deskew(np.array(page))
        text1 = extract_text_from_image(preprocessed_image)
        extracted_text.append(text1)

    # The text extracted is erronous, we combine all of it and then split
    # via spaces and remove unnecessary new line characters and empty strings
    text.extend(extracted_text)
    temp = ""
    for x in text:
      temp += x + " "
    text = [temp]

    text = text[0].split(" ")
    remove = []
    for i in range(len(text)):
      text[i] = re.sub('\n', '', text[i])
      if text[i] == "":
        remove.append(i)
    for i in remove[::-1]:
      text.pop(i)

  return text

In [9]:
# Find all the pdf files in the current directory
pdf_files = [f for f in os.listdir(data_dir) if f.endswith('.pdf')]

# Empty df
df = pd.DataFrame(columns=['file', 'text'])
title = []
texts = []
for pdf_file in pdf_files:
  path = os.path.join(data_dir, pdf_file)
  text = read_prescription(path)
  texts.append(text)
  title.append(pdf_file)

df['file'] = title
df['text'] = texts

LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [8]:
# Extracting medications or advice
keywords = ["medication", "medicine", "advice", "advised" , "food", "diet", "meal"]
medicines = []
for i in range(df.shape[0]):
  prescription = df.iloc[i][1]
  flag = False
  for word in keywords:
    for med in range(len(prescription)):
      if word in prescription[med].lower():
        prescription = prescription[med:]
        flag = True
      if flag:
        break
    if flag:
      break
  medicines.append(" ".join(prescription))

df["extracted"] = medicines

NameError: name 'df' is not defined

In [ ]:
complete = []
for i in range(df.shape[0]):
  prescription = df.iloc[i][1]
  prescription = " ".join(prescription)
  complete.append(prescription)

df["complete"] = complete

In [ ]:
df

In [ ]:
df.iloc[1][2]

# Gemini (A Generative AI Model developed by Google)

In [ ]:
%%capture
!pip install google-generativeai

In [ ]:
#Gemini API: {YOUR_API_KEY_HERE}
import os
import google.generativeai as genai
from google.colab import userdata

os.environ["GEMINI_API_KEY"] = "{YOUR_API_KEY_HERE}"
gemini_key = os.environ["GEMINI_API_KEY"]

# Configure google-generativeai library with the API key
genai.configure(api_key = gemini_key)

In [ ]:
# Supported models for generateContent method
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

In [ ]:
# Simple Test for Gemini
from IPython.display import Markdown
model = genai.GenerativeModel('gemini-pro')

response = model.generate_content("Can you provide medical details of say DOLO tablets.")

Markdown(response.text) #display text as Markdown

In [ ]:
# Prescription Test for Gemini
from IPython.display import Markdown, display

model = genai.GenerativeModel('gemini-pro')

for i in range(df.shape[0]):

  print(f"Below Prescription: {df.iloc[i][0]}")

  response = model.generate_content(df.iloc[i][2] + ". Please identify medicines and their associated symptoms from this prescription, also identify associated diseases. Based on this give Food Recommendations.")

  display(Markdown(response.text))

In [ ]:
# Prescription Test for Gemini
from IPython.display import Markdown, display

model = genai.GenerativeModel('gemini-pro')

for i in range(df.shape[0]):

  print(f"Below Prescription: {df.iloc[i][0]}")

  response = model.generate_content(df.iloc[i][3] + ". Please identify medicines and their associated symptoms from this prescription, also identify associated diseases.  Based on this give Food Recommendations. Structure response in simple table format")

  display(Markdown(response.text))

In [ ]:
response.candidates[0].content.parts[0].text

In [ ]:
def gemini_prompt(prompt):
  import os
  import google.generativeai as genai
  from google.colab import userdata

  os.environ["GEMINI_API_KEY"] = "{YOUR_API_KEY_HERE}"
  gemini_key = os.environ["GEMINI_API_KEY"]

  genai.configure(api_key = gemini_key)
  model = genai.GenerativeModel('gemini-pro')
  response = model.generate_content(prompt)
  return response

# Llama (Large Language Model Meta AI)

In [ ]:
%%capture
!pip install llamaapi -q

In [ ]:
from llamaapi import LlamaAPI
import json

# Replace 'Your_API_Token' with your actual API token
llama = LlamaAPI('{YOUR_API_KEY_HERE}')

In [ ]:
# Test
# JSON object representing the API request
api_request_json = {
  "model": "llama3-70b", # model getting used
  "messages": [
    {"role": "system", "content": "You are a llama assistant that talks like a llama, starting every word with 'll'."}, #system message
    {"role": "user", "content": "Hi, happy llama day!"}, #user message
  ]
}

# Make your request and handle the response
response = llama.run(api_request_json)
print(json.dumps(response.json(), indent=2))

In [ ]:
# Prescription test
api_request_json = {
  "model": "llama3-70b",
  "messages": [
    {"role": "system", "content": "Please provide information about different medicines, their symptoms and their associated diseases from the prescription."},
    {"role": "user", "content": df.iloc[0][2]},
    {"role": "user", "content": df.iloc[1][2]},
    {"role": "user", "content": df.iloc[2][2]},
    {"role": "user", "content": df.iloc[3][2]}
  ]
}

# Make your request and handle the response
response = llama.run(api_request_json)

In [ ]:
response = response.json()["choices"][0]["message"]["content"]
Markdown(response)

# Complete Pipeline

In [ ]:
def get_diet_recommendation():
  from google.colab import files
  from IPython.display import Markdown, display
  import time
  # Ask the user to upload a file
  print("Please upload a PDF file:")
  uploaded_file = files.upload()

  # Check if a file was uploaded
  if len(uploaded_file) > 0:
      file_name = next(iter(uploaded_file))
      if file_name.endswith(".pdf"):
          print("File uploaded successfully:", file_name)
      else:
          print("The uploaded file is not a PDF.")
  else:
      print("No file uploaded.")

  print("Reading Text...")
  ocr = read_prescription("/content/" + file_name)
  ocr = " ".join(ocr)
  print("Identifying Diseases...")
  response = gemini_prompt(ocr + ". Identify medicines in this prescription, then find which disease is associated with them. The give food recommendations to help with the diseases. Give associated disease and food recommendation in tabular format")
  print("Generating Diet Recommendations...")
  display(Markdown(response.text))
  return

In [ ]:
get_diet_recommendation()

# Did not Work

In [ ]:
"""%%capture
!pip install transformers bitsandbytes
!pip install git+https://github.com/huggingface/accelerate"""

In [ ]:
"""from transformers import AutoTokenizer, LlamaTokenizer, LlamaForCausalLM
import torch

model_id = "chaoyi-wu/PMC_LLAMA_7B"

tokenizer = LlamaTokenizer.from_pretrained(model_id, use_fast=True)

model = LlamaForCausalLM.from_pretrained(model_id, pad_token_id=tokenizer.eos_token_id, device_map="auto", load_in_4bit=True)"""

In [ ]:
"""import transformers
import torch
tokenizer = transformers.LlamaTokenizer.from_pretrained('chaoyi-wu/PMC_LLAMA_7B')
model = transformers.LlamaForCausalLM.from_pretrained('chaoyi-wu/PMC_LLAMA_7B', pad_token_id=tokenizer.eos_token_id)
model.cuda()  # move the model to GPU"""